# Import librerie

Mounto google drive

In [ ]:
from google.colab import drive
import zipfile

drive.mount('/content/gdrive')

Mounted at /content/gdrive


Scarico cartella di git sul google drive `oXqTyi2nWP$GRMy`

In [ ]:
!pip install colab_ssh --upgrade
from colab_ssh import init_git
githubUrl = 'https://github.com/khalld/trashbean-classifier.git'
init_git(githubUrl)

Enter your username: (leave it empty if it's 'None')
khalld
Enter your password: 
··········
Successfully cloned the repository in ./trashbean-classifier


In [ ]:
GENERAL_PATHS = {
    'main': '/content/gdrive/MyDrive/trashbean-classifier/',
    'dataset': '/content/gdrive/MyDrive/trashbean-classifier/dataset/',
    'logs': '/content/gdrive/MyDrive/trashbean-classifier/logs/',
    'models': '/content/gdrive/MyDrive/trashbean-classifier/logs/models/',
    'libs': '/content/trashbean-classifier/libs/'
}

librerie custom

In [ ]:
import sys
sys.path.insert(0,GENERAL_PATHS['libs'])

In [ ]:
# need do import only concreteCreator
from TrashbeanDataset import TrashbeanDataset
from TDContainer import TDContainer
from PretrainedModels import PretrainedModelsCreator, CCAlexNet, CCSqueezeNet, CCVgg16
from Training import AverageValueMeter, trainval_classifier, test_classifier, accuracy_score
from utils import get_model_name, import_dataset, split_train_val_test

librerie comuni

In [ ]:
import random
import numpy as np
import pandas as pd
from torchvision import transforms
from os.path import join
from PIL import Image

In [ ]:
random.seed(1996)
np.random.seed(1996)

# Import del dataset

Unzip dell'archivio contente il dataset

In [ ]:
!unzip '/content/gdrive/MyDrive/trashbean-classifier/images.zip' -d '/content/gdrive/MyDrive/trashbean-classifier/dataset/'

In [ ]:
dst_txt = np.loadtxt(join(GENERAL_PATHS['dataset'], 'all_labels.txt'), dtype=str, delimiter=',')

print( len(dst_txt) , dst_txt.shape, "filename", dst_txt[0][0], "classe", dst_txt[0][1])

8100 (8100, 2) filename trashbean_0.jpg classe  0


In [ ]:
images = []
labels = []

for i in range(len(dst_txt)):
  images.append(join(GENERAL_PATHS['dataset'], 'images/') + dst_txt[i][0])  # immagine
  labels.append(dst_txt[i][1])  # classe di appartenenza  
  
dst_df = pd.DataFrame({'image': images, 'label': labels}) # assegno un dataframe per ogni tipo di training

testo il corretto caricamento

In [ ]:
print(dst_df['label'][788])
Image.open(dst_df['image'][788])

In [ ]:
dst_df.head()

,image,label
0,/content/gdrive/MyDrive/trashbean-classifier/d...,0
1,/content/gdrive/MyDrive/trashbean-classifier/d...,0
2,/content/gdrive/MyDrive/trashbean-classifier/d...,0
3,/content/gdrive/MyDrive/trashbean-classifier/d...,0
4,/content/gdrive/MyDrive/trashbean-classifier/d...,0


In [ ]:
training_df, validation_df, test_df = split_train_val_test(dst_df)

Mi assicuro che la distribuzione delle classi sia equa

In [ ]:
training_df['label'].value_counts()

 1    1636
 0    1622
 2    1602
Name: label, dtype: int64

In [ ]:
validation_df['label'].value_counts()

 2    279
 0    272
 1    259
Name: label, dtype: int64

In [ ]:
test_df['label'].value_counts()

 2    819
 0    806
 1    805
Name: label, dtype: int64

Salvo data frame su csv per riutilizzo

In [ ]:
training_df.to_csv(join(GENERAL_PATHS['dataset'], 'training.csv'), index=None)
validation_df.to_csv(join(GENERAL_PATHS['dataset'], 'validation.csv'), index=None)
test_df.to_csv(join(GENERAL_PATHS['dataset'], 'test.csv'), index=None)

In [ ]:
dst_df.to_csv(join(GENERAL_PATHS['dataset'], 'all_dataset.csv'), index=None)

Salvo corrispondenza tra classi in un altro CSV

In [ ]:
ids, classes = zip(*{
    0: "empty",
    1: "half",
    2: "full"
}.items())
ids = pd.DataFrame({'id':ids, 'class':classes}).set_index('id')
ids.to_csv(join(GENERAL_PATHS['dataset'], 'classes.csv'))

# Calcolo media e deviazione standard

In [ ]:
trashbean_dst_all = TrashbeanDataset(join(GENERAL_PATHS['dataset'], 'all_dataset.csv'))

In [ ]:
trashbean_dst_all.__len__()

8100

Calcolo media e deviazione standard

In [ ]:
means = np.zeros(3)
stdevs = np.zeros(3)

for data in trashbean_dst_all:
  img = data[0]
  for i in range(3):
    img = np.asarray(img)
    means[i] += img[i, :, :].mean()
    stdevs[i] += img[i, :, :].std()

means = np.asarray(means) / trashbean_dst_all.__len__()
stdevs = np.asarray(stdevs) / trashbean_dst_all.__len__()
print("{} : normMean = {}".format(type, means))
print("{} : normstdevs = {}".format(type, stdevs))

<class 'type'> : normMean = [127.78855176 127.801383   127.83543153]
<class 'type'> : normstdevs = [36.59095841 36.41266886 36.47179106]


valori di output

normMean = [127.78855176 127.801383   127.83543153]

normstdevs = [36.59095841 36.41266886 36.47179106]

# Loading dataset

In [ ]:
mean_pretrained = [0.485, 0.456, 0.406]
std_pretrained = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    ##transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomPerspective(p=0.3)
    transforms.ToTensor(),
    transforms.Normalize(mean_pretrained, std_pretrained)
    ])

test_transform = transforms.Compose([
    transforms.Resize(256), 
    transforms.CenterCrop(224), # crop centrale
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean_pretrained, std_pretrained)
    ])

In [ ]:
trashbean_dataset = import_dataset(GENERAL_PATHS['dataset'], train_transform=train_transform, test_transform=test_transform)

# https://ai.stackexchange.com/questions/8560/how-do-i-choose-the-optimal-batch-size
trashbean_dataset.create_data_loader(batch_size=32, num_workers=2)

In [ ]:
for sample in trashbean_dataset.training:
  print("image %s label %d" % (sample[0].shape, sample[1]) )
  break

image torch.Size([3, 224, 224]) label 2


# Test su modelli pretrained con finetuning

In [ ]:
def do_training(creator: PretrainedModelsCreator,
                dataset: TDContainer,
                output_class: int,
                model_name: str,
                batch_size: int,
                num_workers: int,
                drop_last: bool,
                lr: float,
                epochs: int,
               ) -> None:
  
  print('Instantiating %s' % (model_name))
  creator.initialize_dst(dataset, output_class, batch_size=batch_size, num_workers=num_workers, drop_last=drop_last)

  print('*** Starting procedure ***')
  model_finetuned = trainval_classifier(model=creator.model, dst_container=dataset, model_name=model_name,
                                        lr=lr, epochs=epochs,
                                        logdir=GENERAL_PATHS['logs'], model_dir=GENERAL_PATHS['models'], save_each_iter=5)
  print("**** Start to calculate accuracy ...")
  model_finetuned_predictions_test, dataset_labels_test = test_classifier(model_finetuned, dataset.test_loader)
  print("**** Accuracy of %s %0.2f%%" % (model_name, accuracy_score(dataset_labels_test, model_finetuned_predictions_test)*100) )

Test VGG16

In [ ]:
do_training(creator=CCVgg16(),
            dataset=trashbean_dataset,
            output_class=3,
            model_name=get_model_name("CCVgg16", "lr=0.001"),
            batch_size=64,
            num_workers=2,
            drop_last=False,
            lr=0.001,
            epochs=50
            )

Test AlexNet

In [ ]:
do_training(creator=CCAlexNet(),
            dataset=trashbean_dataset,
            output_class=3,
            model_name=get_model_name("CCVgg16", "lr=0.001"),
            batch_size=64,
            num_workers=2,
            drop_last=False,
            lr=0.001,
            epochs=50
            )

In [ ]:
# test modello da 0